In [16]:
# 3. Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import os

In [17]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import Image

In [19]:
# 4. Set paths (change this)
DATA_DIR = "/content/drive/MyDrive/24NovDATASET/dataset"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [20]:
CLASSES_DIR = DATA_DIR
classes = os.listdir(CLASSES_DIR)
print("Classes found:", classes)

Classes found: ['watermelon', 'banana', 'Mango', 'cotton']


In [21]:
class_counts = {}
for c in classes:
    class_counts[c] = len(os.listdir(os.path.join(CLASSES_DIR, c)))
print("Class Distribution:", class_counts)

Class Distribution: {'watermelon': 4, 'banana': 2, 'Mango': 8, 'cotton': 4}


In [22]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    classes=classes # Explicitly provide the class names
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    classes=classes # Explicitly provide the class names
)

Found 7522 images belonging to 4 classes.
Found 1879 images belonging to 4 classes.


In [23]:
# 6. Build Model with MobileNetV2
base_model = MobileNetV2(input_shape=IMAGE_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Explicitly build the model to create variables before compiling
model.build(input_shape=(None,) + IMAGE_SIZE + (3,))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Ensure consistent steps
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE, # Ensure consistent steps
    epochs=EPOCHS
)

Epoch 1/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 4524s 19s/step - accuracy: 0.9420 - loss: 0.1673 - val_accuracy: 0.9116 - val_loss: 0.7925
Epoch 2/10
  1/235 ━━━━━━━━━━━━━━━━━━━━ 5:14 1s/step - accuracy: 1.0000 - loss: 0.0032

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


235/235 ━━━━━━━━━━━━━━━━━━━━ 137s 578ms/step - accuracy: 1.0000 - loss: 0.0032 - val_accuracy: 0.9127 - val_loss: 0.8020
Epoch 3/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 687s 3s/step - accuracy: 0.9972 - loss: 0.0101 - val_accuracy: 0.9149 - val_loss: 0.6342
Epoch 4/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 124s 525ms/step - accuracy: 1.0000 - loss: 0.0211 - val_accuracy: 0.9138 - val_loss: 0.6487
Epoch 5/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 638s 3s/step - accuracy: 0.9980 - loss: 0.0102 - val_accuracy: 0.9052 - val_loss: 0.7264
Epoch 6/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 133s 561ms/step - accuracy: 1.0000 - loss: 1.5370e-04 - val_accuracy: 0.9079 - val_loss: 0.7395
Epoch 7/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 654s 3s/step - accuracy: 0.9976 - loss: 0.0061 - val_accuracy: 0.9138 - val_loss: 0.9224
Epoch 8/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 133s 561ms/step - accuracy: 1.0000 - loss: 3.2711e-04 - val_accuracy: 0.9143 - val_loss: 0.9206
Epoch 9/10
235/235 ━━━━━━━━━━━━━━━━━━━━ 608s 3s/step - accuracy: 0.9995 - loss: 0.0019 -

In [25]:
model.save("image_classification_model.h5")


In [26]:
model.save("image_classification_model.keras")
